In [1]:
import os
from typing import Optional
from deepeval.models import DeepEvalBaseLLM
from openai import OpenAI, AsyncOpenAI

class SGlangModel(DeepEvalBaseLLM):
    """
    Custom DeepEval LLM wrapper for a model served via vLLM
    with an OpenAI-compatible API.
    """
    def __init__(self, model_name: str, base_url: str, api_key: Optional[str] = None):
        self.model_name = model_name
        self.base_url = base_url
        self.api_key = api_key if api_key is not None else os.getenv("OPENAI_API_KEY", "123")
        self._sync_client: Optional[OpenAI] = None
        self._async_client: Optional[AsyncOpenAI] = None

    def load_model(self) -> OpenAI:
        if self._sync_client is None:
            self._sync_client = OpenAI(base_url=self.base_url, api_key=self.api_key)
        return self._sync_client
        
    def load_async_model(self) -> AsyncOpenAI:
         if self._async_client is None:
             self._async_client = AsyncOpenAI(base_url=self.base_url, api_key=self.api_key)
         return self._async_client


    def generate(self, prompt: str) -> str:
        client = self.load_model()
        try:
            # Выполняем синхронный вызов к эндпоинту vLLM
            response = client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=150 
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error during synchronous VLLM generation for prompt '{prompt[:50]}...': {e}")
            return "" 

    async def a_generate(self, prompt: str) -> str:
        client = self.load_async_model()
        try:
            response = await client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=150 
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error during asynchronous VLLM generation for prompt '{prompt[:50]}...': {e}")
            return "" 

    def get_model_name(self) -> str:
        """
        Returns the name of the model.
        """
        return self.model_name

In [2]:
from deepeval.models import DeepEvalBaseEmbeddingModel
from typing import List

class InfinityEmbeddingModel(DeepEvalBaseEmbeddingModel):
    def __init__(self, model_name: str, base_url: str, api_key: Optional[str] = None):
        self.model_name = model_name
        self.base_url = base_url
        self.api_key = api_key if api_key is not None else os.getenv("OPENAI_API_KEY", "123")
        self._sync_client: Optional[OpenAI] = None
        self._async_client: Optional[AsyncOpenAI] = None

    def load_model(self) -> OpenAI:
        if self._sync_client is None:
            self._sync_client = OpenAI(base_url=self.base_url, api_key=self.api_key)
        return self._sync_client

    def load_async_model(self) -> AsyncOpenAI:
        if self._async_client is None:
            self._async_client = AsyncOpenAI(base_url=self.base_url, api_key=self.api_key)
        return self._async_client

    def embed_text(self, text: str) -> List[float]:
        client = self.load_model()
        try:
            response = client.embeddings.create(
                model=self.model_name,
                input=text
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"Error during synchronous Infinity embedding for text '{text[:50]}...': {e}")
            return []

    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        client = self.load_model()
        try:
            response = client.embeddings.create(
                model=self.model_name,
                input=texts
            )
            return [data.embedding for data in response.data]
        except Exception as e:
            print(f"Error during synchronous Infinity batch embedding: {e}")
            return [[] for _ in texts] 

    async def a_embed_text(self, text: str) -> List[float]:
        client = self.load_async_model()
        try:
            response = await client.embeddings.create(
                model=self.model_name,
                input=text
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"Error during asynchronous Infinity embedding for text '{text[:50]}...': {e}")
            return []

    async def a_embed_texts(self, texts: List[str]) -> List[List[float]]:
        client = self.load_async_model()
        try:
            response = await client.embeddings.create(
                model=self.model_name,
                input=texts
            )
            return [data.embedding for data in response.data]
        except Exception as e:
            print(f"Error during asynchronous Infinity batch embedding: {e}")
            return [[] for _ in texts]


    def get_model_name(self) -> str:
        """
        Returns the name of the embedding model.
        """
        return self.model_name

## deepcogito/cogito-v1-preview-llama-8B

In [3]:
CogitoLLM = SGlangModel(model_name="deepcogito/cogito-v1-preview-llama-8B", base_url="http://85.143.167.11:30000/v1")

In [4]:
BertaEmbeddings = InfinityEmbeddingModel(model_name="sergeyzh/BERTA", base_url="http://127.0.0.1:7997")

In [5]:
from deepeval.synthesizer import Synthesizer
from deepeval.synthesizer.config import StylingConfig
from deepeval.synthesizer.config import ContextConstructionConfig

styling_config = StylingConfig(
  input_format="Вопросы на русском языке, сформулированные на основе информации из медицинских учебников.",
  task="Генерация вопросов на русском языке для проверки знаний или изучения материала медицинских учебников.",
  scenario="Сценарий: студент или специалист работает с медицинскими учебниками и генерирует вопросы для самопроверки или создания тестов.",
)

synthesizer = Synthesizer(model = CogitoLLM,
                          styling_config = styling_config)

In [ ]:
context_construction_config = ContextConstructionConfig(embedder = BertaEmbeddings,
                                                        critic_model = CogitoLLM,
                                                        context_quality_threshold = 0.1) 

synthesizer.generate_goldens_from_docs(
    document_paths=['/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/books/Kapandzhi_-_Pozvonochnik-276-284.pdf'],
    context_construction_config=context_construction_config)

✨ 🚀 ✨ Loading Documents:   0%|          | 0/1 [00:00<?, ?it/s]

✨ 🧩 ✨ Generating Contexts: 100%|██████████| 9/9 [00:02<00:00,  3.82it/s]


OSError: [Errno 39] Directory not empty: 'da1f2d76-4c57-4417-8605-3ab5848ff81d'

In [7]:
data

NameError: name 'data' is not defined